<img src="ubiops_logo.svg" width="80">

# Quickstart MNIST demo
Deploy your deployment in UbiOps using the python client library.


The first step is to [download](https://storage.googleapis.com/ubiops/example-deployment-packages/mnist_deployment_package.zip) a prepared deployment and some sample data ([image1](https://storage.googleapis.com/ubiops/example-deployment-packages/1.jpg), [image2](https://storage.googleapis.com/ubiops/example-deployment-packages/2.jpg), [image3](https://storage.googleapis.com/ubiops/example-deployment-packages/3.jpg)).

In [ ]:
!curl -X GET https://storage.googleapis.com/ubiops/example-deployment-packages/mnist_deployment_package.zip -o mnist_deployment_package.zip
!curl -X GET https://storage.googleapis.com/ubiops/example-deployment-packages/1.jpg -o 1.jpg
!curl -X GET https://storage.googleapis.com/ubiops/example-deployment-packages/2.jpg -o 2.jpg
!curl -X GET https://storage.googleapis.com/ubiops/example-deployment-packages/3.jpg -o 3.jpg

In [ ]:
image_files = ['1.jpg', '2.jpg', '3.jpg']

Add your API token and project name. You can also adapt the deployment name and deployment version name or leave the default values. Afterwards we initialize the client library, which establishes the connection with UbiOps.

In [ ]:
API_TOKEN = 'Token <YOUR_API_KEY>'
PROJECT_NAME = '<YOUR_PROJECT_NAME>'
DEPLOYMENT_NAME = 'mnist-tutorial'
DEPLOYMENT_VERSION = 'v1'

In [ ]:
import ubiops
configuration = ubiops.Configuration(host="https://api.ubiops.com/v2.1")
configuration.api_key['Authorization'] = API_TOKEN

Here we open the connection with the UbiOps API Client.

In [ ]:
client = ubiops.ApiClient(configuration)
api = ubiops.CoreApi(client)
api.service_status()

## Deploy
Create a deploytment for mnist.

In [ ]:
mnist_template = ubiops.DeploymentCreate(
    name=DEPLOYMENT_NAME,
    description='A deployment to classify handwritten digits.',
    input_type='structured',
    output_type='structured',
    input_fields=[
        {'name': 'image', 'data_type': 'blob'}
    ],
    output_fields=[
        {'name': 'prediction', 'data_type': 'int'},
        {'name': 'probability', 'data_type': 'double'}
    ],
    labels={"demo": "mnist"}
)

mnist_deployment = api.deployments_create(project_name=PROJECT_NAME, data=mnist_template)
print(mnist_deployment)

Create a deployment version for mnist.

- Use `request_retention_mode='full'` to store all requests made to the deployment version, so we can retrieve them later in this notebook.

In [ ]:
version_template = ubiops.DeploymentVersionCreate(
    version=DEPLOYMENT_VERSION,
    language='python3.6',
    instance_type='1024mb',
    maximum_instances=1,
    minimum_instances=0,
    maximum_idle_time=1800, # = 30 minutes
    deployment_mode='express',  # 'express' or 'batch'
    request_retention_mode='full',  # input/output of requests will be stored
    request_retention_time=3600  # requests will be stored for 1 hour
)

version = api.deployment_versions_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    data=version_template
)
print(version)

Upload the prepared deployment file to the created version.

In [ ]:
upload_response = api.revisions_file_upload(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION,
    file='mnist_deployment_package.zip'
)
print(upload_response)

Check if deployment is finished building.
This can take a few minutes.

In [ ]:
from time import sleep
status = 'queued'
while status != 'success' and status != 'failed':
    build_status = api.builds_get(
        project_name=PROJECT_NAME,
        deployment_name=DEPLOYMENT_NAME,
        version=DEPLOYMENT_VERSION,
        build_id=upload_response.build
    )
    status = build_status.status
    print("{:15s}".format(status), end='\r')
    sleep(1)
print(status)

The version is now available.

In [ ]:
api.deployment_versions_get(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION
).status

## Create multiple direct requests
Make sure the deployment is in 'available' state before performing deployment requests.

Create a deployment request for each image and get the results directly if the request is finished.

The first request is usually slow because of a cold start. A second request (performed within the `maximum_idle_time`, which was set to 30 minutes in this tutorial) will be much faster.

In [ ]:
from IPython.display import Image, display

for image_file in image_files:
    # Display the input image
    display(Image(url=image_file, width=40, height=40))
    
    # First upload the image
    blob = api.blobs_create(project_name=PROJECT_NAME, file=image_file)
    
    # Make a request using the blob id as input.
    data = {'image': blob.id}
    request_i = api.deployment_version_requests_create(
        project_name=PROJECT_NAME,
        deployment_name=DEPLOYMENT_NAME,
        version=DEPLOYMENT_VERSION,
        data=data
    )
    
    # Or make a request to the default deployment version:
#     request_i = api.deployment_requests_create(
#         project_name=PROJECT_NAME,
#         deployment_name=DEPLOYMENT_NAME,
#         data=data
#     )
    print(request_i.result)

## List all requests
List all requests of the deployment. <br/>

<div class="alert alert-block alert-info">
    
Requests are only stored if you have <code>request_retention_model='full'</code> for your deployment version. They are stored for a limited time (<code>request_retention_time</code> seconds).
</div>

In [ ]:
import pprint

all_requests = api.deployment_version_requests_list(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION
)
pprint.pprint(all_requests)

## Retrieve a request
Retrieve the details of a single request of the deployment. <br/>

In [ ]:
request_1 = api.deployment_version_requests_get(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION,
    request_id=all_requests[0].id
)
print(request_1)

## Cleanup
Delete created deployment.

In [ ]:
api.deployments_delete(project_name=PROJECT_NAME, deployment_name=DEPLOYMENT_NAME)

## Close connection
Close the connection with the UbiOps API client.

In [ ]:
client.close()